In [1]:
import time
import pandas as pd
import numpy as np
import cv2 # import OpenCV
import matplotlib.pyplot as plt
from tqdm import tqdm #pour barre de temps dans une boucle for
from functions.Filtering import OTSU_threshold, filter_color_threshold,filter_Kmeans1,filter_Kmeans2,filter_MeanShift
from functions.storing import database_generate,database_load
from functions.machine_learning import modeling

# from datetime import date

from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestClassifier
from sklearn import svm

In [2]:
df_exp=pd.read_csv('variables/prep_info.csv')
# df_exp.savename.dropna()
df_exp.head()

,who,date,savename,sample_per_cell_type,size,filter,preprocessing_status,preprocessing_time,saving_time,training_status,pca,pca_components,loading_time,train_test_split_time,RF_time,RF_train_acc,RF_test_acc,SVM_time,SVM_train_acc,SVM_test_acc
0,Emilien,17/12/2021,NaN,50.0,360x360,rgb,Done,NaN,836.11,Done,no,NaN,/,73.29,48.27,NaN,0.775,65.91,NaN,0.663
1,Emilien,17/12/2021,NaN,50.0,180x180,rgb,Done,NaN,163.34,Done,no,NaN,36.24,7.07,9.05,NaN,0.75,14.89,NaN,0.663
2,Emilien,17/12/2021,NaN,50.0,90x90,rgb,Done,NaN,18.93,Done,no,NaN,3.08,0.05,1.45,NaN,0.663,2.33,NaN,0.563
3,Emilien,17/12/2021,NaN,300.0,360x360,rgb,Done,NaN,OUT OF MEMORY,Done,no,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Emilien,17/12/2021,NaN,300.0,180x180,rgb,Done,NaN,2080.76,Done,no,NaN,214.06,2.07,107.32,NaN,0.814,238.99,NaN,0.731


In [3]:
df_exp=pd.read_csv('variables/prep_info.csv')
idx=df_exp[df_exp.training_status=='Todo'].index # récupère les indexes des lignes à traiter
# idx=df_exp[df_exp.Training_status=='Todo' & df_exp.savename.isna()==False].index

for i in idx[:2]:
    
    savename = df_exp.loc[i,'savename']
    size = tuple([int(x) for x in df_exp.loc[i,'size'].split('x')])
    print(f'Training database {savename}')

    data_df, elapsed_load = database_load(savename,size)

    df_exp.loc[i,'loading_time']=round(elapsed_load,2)
    
    target = data_df['label'].apply(lambda x: x.split(' - ')[0])
    feats = data_df.iloc[:,2:]

    if df_exp.loc[i,'pca'] == 'yes':
        # AVEC PCA
        pca=PCA(n_components =0.9).fit(feats)
        df_exp.loc[i,'pca_components']=pca.n_components_
        # print('number of pixels kept:',pca.n_components_)
        feats_pca=pca.transform(feats)
        acc_train,acc_test,elapsed_train = modeling(feats_pca,target,svm.SVC(),display_results_list=[])
    else:
        # SANS PCA
        acc_train,acc_test,elapsed_train = modeling(feats,target,svm.SVC(),display_results_list=[])
    
    # Enregistrement des résulats
    df_exp.loc[i,'SVM_time']=round(elapsed_train,2)
    df_exp.loc[i,'SVM_train_acc']=round(acc_train,3)
    df_exp.loc[i,'SVM_test_acc']=round(acc_test,3)
    
    # rf = RandomForestClassifier(max_depth=10, max_leaf_nodes=20) # on limite le nombre de feuilles et la profondeur pour éviter l'overfitting
    rf = RandomForestClassifier(max_depth=10)
    acc_train,acc_test,elapsed_train = modeling(feats_pca,target,rf,display_results_list=[])
    # df2.loc[idx,'train_test_split_time']=elapsed_split
    df2.loc[idx,'RF_time']=round(elapsed_train,2)
    df2.loc[idx,'RF_train_acc']=round(acc_train,3)
    df2.loc[idx,'RF_test_acc']=round(acc_train,3)

    df_exp.loc[i,'training_status']='Done'

    df_exp.to_csv('variables/prep_info.csv',index=False)

Training database variables/data_prep/data_size-90x90_sample-500_f-Kmeans1.csv
Time elapsed to load variables/data_prep/data_size-90x90_sample-500_f-Kmeans1.csv: 0.0min 4.0sec
Time elapsed to train model: 2.0min 30.0sec


NameError: name 'feats_pca' is not defined